In [1]:
import os
os.chdir("/workspace/train/kmb_simcse")

from model import ModernBERTSimCSE
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, Trainer, get_scheduler, TrainingArguments, ModernBertForMaskedLM, ModernBertModel


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = ModernBERTSimCSE.from_pretrained(
    "x2bee/KoModernBERT-base-mlm-v03-retry-ckp03",
    subfolder="last-checkpoint",
    pooler_type="avg",
)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


In [3]:
model.config

ModernBertConfig {
  "_name_or_path": "x2bee/KoModernBERT-base-mlm-v03-retry-ckp03",
  "architectures": [
    "ModernBertForMaskedLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 50281,
  "classifier_activation": "gelu",
  "classifier_bias": false,
  "classifier_dropout": 0.0,
  "classifier_pooling": "mean",
  "cls_token_id": 50281,
  "decoder_bias": true,
  "deterministic_flash_attn": false,
  "embedding_dropout": 0.0,
  "eos_token_id": 50282,
  "global_attn_every_n_layers": 3,
  "global_rope_theta": 160000.0,
  "gradient_checkpointing": false,
  "hidden_activation": "gelu",
  "hidden_size": 768,
  "initializer_cutoff_factor": 2.0,
  "initializer_range": 0.02,
  "intermediate_size": 1152,
  "layer_norm_eps": 1e-05,
  "local_attention": 128,
  "local_rope_theta": 10000.0,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "mlp_dropout": 0.0,
  "model_type": "modernbert",
  "norm_bias": false,
  "norm_eps": 1e-05,
  "num_attention_heads": 12,
  "n

In [7]:
model

ModernBERTSimCSE(
  (modernbert): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(95663, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=7

In [4]:
from transformers import pipeline

pipe = pipeline("feature-extraction", model="CocoRoF/KMB_SimCSE_test")

Some weights of the model checkpoint at CocoRoF/KMB_SimCSE_test were not used when initializing ModernBertModel: ['modernbert.embeddings.norm.weight', 'modernbert.embeddings.tok_embeddings.weight', 'modernbert.final_norm.weight', 'modernbert.layers.0.attn.Wo.weight', 'modernbert.layers.0.attn.Wqkv.weight', 'modernbert.layers.0.mlp.Wi.weight', 'modernbert.layers.0.mlp.Wo.weight', 'modernbert.layers.0.mlp_norm.weight', 'modernbert.layers.1.attn.Wo.weight', 'modernbert.layers.1.attn.Wqkv.weight', 'modernbert.layers.1.attn_norm.weight', 'modernbert.layers.1.mlp.Wi.weight', 'modernbert.layers.1.mlp.Wo.weight', 'modernbert.layers.1.mlp_norm.weight', 'modernbert.layers.10.attn.Wo.weight', 'modernbert.layers.10.attn.Wqkv.weight', 'modernbert.layers.10.attn_norm.weight', 'modernbert.layers.10.mlp.Wi.weight', 'modernbert.layers.10.mlp.Wo.weight', 'modernbert.layers.10.mlp_norm.weight', 'modernbert.layers.11.attn.Wo.weight', 'modernbert.layers.11.attn.Wqkv.weight', 'modernbert.layers.11.attn_norm

In [16]:
score01

tensor([[99.9057]], device='cuda:0', grad_fn=<MulBackward0>)

In [12]:
embeddings[0].shape

torch.Size([3, 16, 768])

In [33]:
import torch

# Tokenizer로 데이터를 토큰화
result = tokenizer(
    "sentence1",
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"  # PyTorch 텐서로 반환
)

# GPU로 텐서 이동 (CUDA 디바이스 사용 가능 여부 확인)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
result = {key: value.to(device) for key, value in result.items()}
model = model.to(device)  # 모델도 GPU로 이동

# 모델에 입력
anchor_pooler = model(
    input_ids=result['input_ids'],
    attention_mask=result['attention_mask'],
    return_dict=True,
)


In [35]:
anchor_pooler.pooler_output

AttributeError: 'BaseModelOutput' object has no attribute 'pooler_output'

In [25]:
anchor_pooler[0][:, 0].shape

torch.Size([1, 768])

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta-multitask")
modelmb = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta-multitask")

In [9]:
modelmb

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
|